In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import pandas as pd
import matplotlib
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from mpl_toolkits.axisartist.axislines import SubplotZero
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)


In [2]:
# matplotlib.rcParams["text.usetex"] = True
# plt.rcParams["figure.figsize"] = [8, 4]
# plt.xlabel('Time', size = 12)
# plt.ylabel('Expectations', size = 12)
# plt.rcParams['figure.dpi'] = 300
# ax = plt.gca()

In [3]:
stmt = 'SELECT DISTINCT(M.id), M.text, M.like_count, M.retweet_count, M.quote_count, M.date, B.sentiment AS s FROM machine_learning M LEFT JOIN base_sentiment B ON M.id = B.id'
df = pd.read_sql(stmt, con=engine)

In [4]:
from datetime import datetime
def sentiment(sentiment):
    if sentiment == 0:
        return 'negative'
    if sentiment == 1:
        return 'neutral'
    if sentiment == 2:
        return 'positive'

def dformat(d):
    return datetime.strptime(d, '%Y-%m-%d').strftime('%Y')
    
df['sentiment'] = df['s'].apply(sentiment)
df.date = df.date.apply(dformat)

In [5]:
df.head()

,id,text,like_count,retweet_count,quote_count,date,s,sentiment
0,8424885080,my 3 mth career break has started now for lots...,0,0,0,2010,2,positive
1,8424527990,prediction of sporting results using neural ne...,0,0,0,2010,1,neutral
2,8420419006,im learning the drawback of selling in a middl...,0,0,0,2010,1,neutral
3,8419386549,encyclopedia of machine learning springerverl...,0,0,0,2010,2,positive
4,8415166607,junior researchermathematical modeller machine...,0,0,0,2010,1,neutral


In [6]:
# matplotlib.rcParams["text.usetex"] = True
# plt.rcParams["figure.figsize"] = [8, 4]
# plt.rcParams['figure.dpi'] = 300
# sns.countplot(x= "date", hue="sentiment", data=df, palette=['#abd0e6',"#6aaed6", '#105ba4'])
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.xlabel('Year', size = 12)
# plt.ylabel('Number of Tweets', size = 12)
# plt.savefig(os.path.join(path, 'sentiment_count.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()

In [7]:
# df['tweet_length'] = [len(t) for t in df.text]

In [8]:
# matplotlib.rcParams["text.usetex"] = True
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# sns.boxplot(y='tweet_length', x='date', 
#                  data=df, 
#                  width=0.5,
#                  palette=['#d6e6f4', '#6aaed6', '#105ba4'],
#                  hue="sentiment",
#                  linewidth=1,
#                  fliersize=1,
#                  ax=ax,)
# plt.rcParams["figure.figsize"] = [8, 4]
# plt.rcParams['figure.dpi'] = 300
# plt.xlabel('Year', size = 12)
# plt.ylabel('Number of Characters per Tweet', size = 12)
# plt.savefig(os.path.join(path, 'tweet_length_year.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()

In [9]:
cvec = CountVectorizer()
cvec.fit(df.text)

CountVectorizer()

In [10]:
len(cvec.get_feature_names_out())

783584

In [11]:
neg_doc_matrix = cvec.transform(df[df.sentiment == 'negative'].text)
neu_doc_matrix = cvec.transform(df[df.sentiment == 'neutral'].text)
pos_doc_matrix = cvec.transform(df[df.sentiment == 'positive'].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
neu_tf = np.sum(neu_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
neu = np.squeeze(np.asarray(neu_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,neu,pos],columns=cvec.get_feature_names_out()).transpose()

In [12]:
term_freq_df.columns = ['negative', 'neutral', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['neutral'] + term_freq_df['positive']
term_freq_df = term_freq_df.sort_values(by='total', ascending=False).iloc[:2000000, :]


In [13]:
term_freq_df.head()

,negative,neutral,positive,total
learning,571223,2741067,1805310,5117600
machine,531710,2632804,1748186,4912700
to,272002,953808,1119135,2344945
the,322937,859244,1142089,2324270
and,359395,925545,1013158,2298098


In [14]:
y_pos = np.arange(500)
plt.figure(figsize=(8,4))
s = 1 #exponential term
expected_zipf = [term_freq_df['total'][0]/(i+1)**s for i in y_pos]

plt.bar(y_pos, term_freq_df['total'][:500], align='center', color = '#003366')
plt.plot(y_pos, expected_zipf, color='#E31B23', linestyle='--',linewidth=2)
plt.ylabel('Frequency', size=12)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(os.path.join(path, 'zipfs.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [15]:
from pylab import *
counts = term_freq_df.total
tokens = term_freq_df.index
ranks = arange(1, len(counts)+1)
indices = argsort(-counts)
frequencies = counts[indices]
plt.figure(figsize=(8,4))
plt.ylim(1,10**7)
plt.xlim(1,10**7)
loglog(ranks, frequencies, marker=".")

plt.plot([1,frequencies[0]],[frequencies[0],1],color='r')
plt.xlabel("Frequency rank of token", size=12)
plt.ylabel("Absolute frequency of token", size=12)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for n in list(logspace(-0.5, log10(len(counts)-2), 25).astype(int)):
    dummy = text(ranks[n], frequencies[n], " " + tokens[indices[n]], 
                 verticalalignment="bottom",
                 horizontalalignment="left")

plt.savefig(os.path.join(path, 'zipf_plot.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [16]:
# y_pos = np.arange(50)
# plt.figure(figsize=(8,4))
# plt.bar(y_pos, term_freq_df.sort_values(by='negative', ascending=False)['negative'][:50], align='center', color = '#003366')
# plt.xticks(y_pos, term_freq_df.sort_values(by='negative', ascending=False)['negative'][:50].index,rotation='vertical')
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.ylabel('Frequency', size =12)
# plt.xlabel('Top 50 negative tokens', size =12)
# plt.savefig(os.path.join(path, 'zipfs_negative.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()


In [17]:
# y_pos = np.arange(50)
# plt.figure(figsize=(8,4))
# plt.bar(y_pos, term_freq_df.sort_values(by='positive', ascending=False)['positive'][:50], align='center', color = '#003366')
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.xticks(y_pos, term_freq_df.sort_values(by='positive', ascending=False)['positive'][:50].index,rotation='vertical')
# plt.ylabel('Frequency', size=12)
# plt.xlabel('Top 50 positive tokens', size=12)
# plt.savefig(os.path.join(path, 'zipfs_positive.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()


In [18]:
plt.figure(figsize=(8,4))
ax = sns.regplot(x="negative", y="positive",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df)
plt.ylabel('Positive Frequency', size=12)
plt.xlabel('Negative Frequency', size=12)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(os.path.join(path, 'neg_vs_positive.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()


In [19]:
term_freq_df['pos_rate'] = term_freq_df['positive'] * 1./term_freq_df['total']
term_freq_df.sort_values(by='pos_rate', ascending=False).iloc[:10]

,negative,neutral,positive,total,pos_rate
onvehicle,0,0,1,1,1.0
hj9prhcbv,0,0,2,2,1.0
cyberbite,0,0,1,1,1.0
idbringing,0,0,2,2,1.0
cyberbattleskim,0,0,1,1,1.0
cyberawards2020,0,0,1,1,1.0
pseudonymised,0,0,2,2,1.0
hix,0,0,2,2,1.0
pseudonormsseminorms,0,0,2,2,1.0
cyberauctions,0,0,1,1,1.0


In [20]:
term_freq_df['pos_freq_pct'] = term_freq_df['positive'] * 1./term_freq_df['positive'].sum()
term_freq_df.sort_values(by='pos_freq_pct', ascending=False).iloc[:10]

,negative,neutral,positive,total,pos_rate,pos_freq_pct
learning,571223,2741067,1805310,5117600,0.352765,0.044215
machine,531710,2632804,1748186,4912700,0.355850,0.042816
the,322937,859244,1142089,2324270,0.491375,0.027971
to,272002,953808,1119135,2344945,0.477254,0.027409
and,359395,925545,1013158,2298098,0.440868,0.024814
of,195707,572792,659279,1427778,0.461752,0.016147
in,178629,613560,653893,1446082,0.452183,0.016015
for,130377,540679,603030,1274086,0.473304,0.014769
ai,178844,713041,588371,1480256,0.397479,0.014410
machinelearning,123625,816574,502923,1443122,0.348497,0.012317


In [21]:
from scipy.stats import hmean
term_freq_df['pos_hmean'] = term_freq_df.apply(lambda x: (hmean([x['pos_rate'], x['pos_freq_pct']])                                                               if x['pos_rate'] > 0 and x['pos_freq_pct'] > 0 else 0), axis=1)
                                                       
term_freq_df.sort_values(by='pos_hmean', ascending=False).iloc[:10]

,negative,neutral,positive,total,pos_rate,pos_freq_pct,pos_hmean
learning,571223,2741067,1805310,5117600,0.352765,0.044215,0.078580
machine,531710,2632804,1748186,4912700,0.355850,0.042816,0.076435
the,322937,859244,1142089,2324270,0.491375,0.027971,0.052930
to,272002,953808,1119135,2344945,0.477254,0.027409,0.051841
and,359395,925545,1013158,2298098,0.440868,0.024814,0.046983
of,195707,572792,659279,1427778,0.461752,0.016147,0.031202
in,178629,613560,653893,1446082,0.452183,0.016015,0.030934
for,130377,540679,603030,1274086,0.473304,0.014769,0.028644
ai,178844,713041,588371,1480256,0.397479,0.014410,0.027812
machinelearning,123625,816574,502923,1443122,0.348497,0.012317,0.023794


In [22]:
from scipy.stats import norm
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df['pos_rate_normcdf'] = normcdf(term_freq_df['pos_rate'])
term_freq_df['pos_freq_pct_normcdf'] = normcdf(term_freq_df['pos_freq_pct'])
term_freq_df['pos_normcdf_hmean'] = hmean([term_freq_df['pos_rate_normcdf'], term_freq_df['pos_freq_pct_normcdf']])
term_freq_df.sort_values(by='pos_normcdf_hmean',ascending=False).iloc[:10]

,negative,neutral,positive,total,pos_rate,pos_freq_pct,pos_hmean,pos_rate_normcdf,pos_freq_pct_normcdf,pos_normcdf_hmean
best,207,104,85986,86297,0.996396,0.002106,0.004203,0.891127,1.000000,0.942429
awesome,13,14,12812,12839,0.997897,0.000314,0.000627,0.891762,0.998899,0.942295
successful,66,0,11847,11913,0.994460,0.000290,0.000580,0.890303,0.997673,0.940935
intelligent,235,100,30719,31054,0.989212,0.000752,0.001504,0.888048,1.000000,0.940705
great,892,44,71491,72427,0.987077,0.001751,0.003496,0.887121,1.000000,0.940185
amazing,301,2,21328,21631,0.985992,0.000522,0.001044,0.886649,1.000000,0.939919
popular,134,114,14694,14942,0.983402,0.000360,0.000719,0.885514,0.999779,0.939183
nice,91,129,12829,13049,0.983140,0.000314,0.000628,0.885398,0.998914,0.938737
available,689,48,30809,31546,0.976637,0.000755,0.001508,0.882511,1.000000,0.937589
euro,32,80,10086,10198,0.989017,0.000247,0.000494,0.887964,0.991971,0.937090


In [23]:
term_freq_df['neg_rate'] = term_freq_df['negative'] * 1./term_freq_df['total']
term_freq_df['neg_freq_pct'] = term_freq_df['negative'] * 1./term_freq_df['negative'].sum()
term_freq_df['neg_hmean'] = term_freq_df.apply(lambda x: (hmean([x['neg_rate'], x['neg_freq_pct']])                                                                if x['neg_rate'] > 0 and x['neg_freq_pct'] > 0                                                                else 0), axis=1)
                                                       
term_freq_df['neg_rate_normcdf'] = normcdf(term_freq_df['neg_rate'])
term_freq_df['neg_freq_pct_normcdf'] = normcdf(term_freq_df['neg_freq_pct'])
term_freq_df['neg_normcdf_hmean'] = hmean([term_freq_df['neg_rate_normcdf'], term_freq_df['neg_freq_pct_normcdf']])
term_freq_df.sort_values(by='neg_normcdf_hmean', ascending=False).iloc[:10]

,negative,neutral,positive,total,pos_rate,pos_freq_pct,pos_hmean,pos_rate_normcdf,pos_freq_pct_normcdf,pos_normcdf_hmean,neg_rate,neg_freq_pct,neg_hmean,neg_rate_normcdf,neg_freq_pct_normcdf,neg_normcdf_hmean
random,10003,405,1064,11472,0.092748,0.000026,0.000052,0.205713,0.595919,0.305847,0.871949,0.000873,0.001744,0.995044,1.000000,0.997516
artificial,265294,2272,39620,307186,0.128977,0.000970,0.001926,0.229942,1.000000,0.373907,0.863627,0.023154,0.045099,0.994617,1.000000,0.997301
fake,7885,362,1006,9253,0.108721,0.000025,0.000049,0.216204,0.590519,0.316521,0.852156,0.000688,0.001375,0.993975,1.000000,0.996978
plain,3532,29,345,3906,0.088326,0.000008,0.000017,0.202864,0.528014,0.293113,0.904250,0.000308,0.000616,0.996430,0.996728,0.996579
wrong,6942,313,1196,8451,0.141522,0.000029,0.000059,0.238689,0.608138,0.342823,0.821441,0.000606,0.001211,0.991909,1.000000,0.995938
cutting,4189,94,940,5223,0.179973,0.000023,0.000046,0.266600,0.584354,0.366151,0.802029,0.000366,0.000731,0.990304,0.999375,0.994819
mean,11877,4,3217,15098,0.213075,0.000079,0.000158,0.291883,0.776193,0.424235,0.786660,0.001037,0.002070,0.988841,1.000000,0.994389
intelligencemachine,3317,148,419,3884,0.107878,0.000010,0.000021,0.215643,0.535075,0.307399,0.854016,0.000289,0.000579,0.994083,0.994661,0.994372
hidden,5467,4,1592,7063,0.225400,0.000039,0.000078,0.301575,0.644118,0.410810,0.774034,0.000477,0.000954,0.987500,0.999988,0.993704
bad,9264,1,2736,12001,0.227981,0.000067,0.000134,0.303623,0.740207,0.430614,0.771936,0.000809,0.001615,0.987264,1.000000,0.993591


In [24]:
plt.figure(figsize=(8,4))
ax = sns.regplot(x="neg_hmean", y="pos_hmean",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel('Positive Rate and Frequency Harmonic Mean', size=12)
plt.xlabel('Negative Rate and Frequency Harmonic Mean', size=12)
plt.savefig(os.path.join(path, 'neghmean_vs_poshmean.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()


In [ ]:
plt.figure(figsize=(8,4))
ax = sns.regplot(x="neg_normcdf_hmean", y="pos_normcdf_hmean",fit_reg=False, scatter_kws={'alpha':0.5},data=term_freq_df)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel('Positive Rate and Frequency CDF Harmonic Mean',size = 12)
plt.xlabel('Negative Rate and Frequency CDF Harmonic Mean',size =12)
plt.savefig(os.path.join(path, 'normcdf_hmean.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()